### PRUEBA CON EL MODELO FINAL PARA LA PUESTA EN PRODUCCION CON LOS DATOS DE TEST NO UTILIZADOS HASTA AHORA



In [1]:
import pandas as pd
import numpy as np
import joblib
import sys
sys.path.append("C:/Users/Fernando/Desktop/Proyecto_Final_ML/src")
import preprocess_functions as pr
import funciones_clases_pipeline_produccion

In [14]:
data_test = pd.read_csv("C:/Users/Fernando/Desktop/Proyecto_Final_ML/data/data_test.csv", index_col=0)

In [15]:
# MODIFICACION DE LA COLUMNA Y (target)
data_test = data_test[~data_test["Company response"].isin(["In progress", "Untimely response"])]
# Creación del diccionario con los valores para codificar la columna Company reponse.
estado_respuesta = {'Closed with monetary relief': 0, 
       'Closed with non-monetary relief':0, 'Closed with explanation': 1, 'Closed': 1}

data_test["Company response"] = data_test["Company response"].map(estado_respuesta)
X = data_test.drop(columns=["Company response"])
y = data_test["Company response"]


In [8]:
# Cargamos el pipeline

pipeline = joblib.load("C:/Users/Fernando/Desktop/Proyecto_Final_ML/pipelines/pipeline_produccion.pkl")

In [18]:
y_pred = pipeline.predict(X)
y_proba = pipeline.predict_proba(X)

In [19]:
from sklearn.metrics import classification_report, roc_auc_score
print(classification_report(y, y_pred))
print(roc_auc_score(y, y_proba[:, 1]))

              precision    recall  f1-score   support

           0       0.28      0.75      0.41      1070
           1       0.88      0.49      0.63      4000

    accuracy                           0.54      5070
   macro avg       0.58      0.62      0.52      5070
weighted avg       0.75      0.54      0.58      5070

0.6420413551401869


## Modelo muy robusto ya que los valores en las métricas obtenidos para el modelo con los datos de validacion no expuestos anteriormente presentan valores similares a los de entrenamiento y test

## DESPLIEGUE CON GRADIO

In [ ]:
import joblib
import gradio as gr
import pandas as pd

In [ ]:

pipeline = joblib.load("C:/Users/Fernando/Desktop/Proyecto_Final_ML/pipelines/pipeline_produccion.pkl")

def predecir_modelo(Complaint_ID, Product, sub_product, issue, sub_issue, state,
       zip_code, date_received, date_sent_to_company, company,
       timely_response = 0, consumer_disputed = 0):
    # Crear un DataFrame con la entrada del usuario
    X_new = pd.DataFrame([{'Complaint ID': Complaint_ID,'Product': Product, 'Sub-product': sub_product,'Issue': issue, 'Sub-issue': sub_issue,
                           'State': state, 'ZIP code': zip_code,'Date received': date_received,
                           'Date sent to company': date_sent_to_company,
                           'Company': company,'Timely response?': timely_response,'Consumer disputed?': consumer_disputed}])
    
    # Predecir con pipeline
    pred = pipeline.predict(X_new)
    prob = pipeline.predict_proba(X_new)[:,1][0]  # probabilidad de clase positiva
    
    return f"Predicción: {pred[0]}", f"Probabilidad clase positiva: {prob:.2f}"

